In [46]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [47]:
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"

In [49]:
#Load dos modelos (Embedding e LLM)

embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", max_tokens=300)

In [50]:
#Carregar o PDF

pdf_link = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [52]:
#Separar em Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000, 
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [53]:
#Salvar os Chunks no VectorDB

if os.path.exists("naive"):
    vectorDB = Chroma(
        embedding_function=embedding_model, 
        persist_directory="naive"
    )
    
# Se não existir, crie o banco de dados e salve os documentos
else:
    vectorDB = Chroma.from_documents(
        documents=chunks, 
        embedding=embedding_model, 
        persist_directory="naive"
    )

In [54]:
naive_retriever = vectorDB.as_retriever(search_kwargs={"k": 10})


In [ ]:
os.environ["CO_API_KEY"] = "api_key"

rerank = CohereRerank(
    model="rerank-v3.5",
    top_n=3
)

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retriever
)

In [56]:
TEMPLATE = """"
    Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado.
    Contexto:
    {context}

    Pergunta:
    {question}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [57]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})
output_parser = StrOutputParser()
compressor_retrieval_chain = setup_retrieval | rag_prompt | llm | output_parser

In [59]:
# Pergunta a serem respondidas sobre "Os Sertões" de Euclides da Cunha:


questions = [
"Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
"Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
"Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
"Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
" Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",

] 


for question in questions:    
    print(f"Pergunta: {question}")
    print("Resposta:", compressor_retrieval_chain.invoke(question))
    print("\n" + "="*50 + "\n")

Pergunta: Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?
Resposta: A visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino em *Os Sertões* é complexa e ambivalente,  longe de uma simples descrição romântica ou determinista.  Ele não o vê como mero cenário, mas como um personagem fundamental, um ator crucial na tragédia humana que se desenrola.

Euclides descreve o sertão com uma precisão geográfica e científica impressionante, detalhando a vegetação (caatinga), o clima árido e inóspito, a escassez de água e a sua distribuição irregular.  Ele utiliza uma linguagem rica em detalhes,  explorando a aspereza da paisagem, a força bruta da natureza e a sua beleza paradoxal, muitas vezes hostil, porém fascinante.  A descrição minuciosa da geografia, da flora e da fauna,  se transforma numa poderosa ferramenta para compreender a vida e a cultura sertaneja.

A influência do ambiente na vida dos hab